In [3]:
from pyspark.sql import SparkSession,Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
import urllib.request
import json

In [5]:
spark=SparkSession.builder.\
    appName("Test 3").\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/26 15:21:16 INFO SparkEnv: Registering MapOutputTracker
24/06/26 15:21:16 INFO SparkEnv: Registering BlockManagerMaster
24/06/26 15:21:16 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/06/26 15:21:17 INFO SparkEnv: Registering OutputCommitCoordinator


In [ ]:
url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"
file_path = "/tmp/chipotle.tsv"
urllib.request.urlretrieve(url, file_path)

In [ ]:
pd1=spark.read.\
    format("csv").\
        option("header","true").\
            option("inferschema","true").\
                option("sep","\t").\
                    load("/user/sovik/chipotle.tsv")

In [ ]:
#first 10 datas

pd1.show(10,truncate=False)

In [ ]:
#no of count

pd1.count()

In [ ]:
#count of columns
len(pd1.columns)

In [ ]:
pd1.columns

In [ ]:
#check index
# Add an index column
pd1_with_index = pd1.withColumn("index", monotonically_increasing_id())

# Show the DataFrame schema
pd1_with_index.printSchema()


In [ ]:
# Show the first few rows
pd1_with_index.show(10)

In [ ]:
pd2=pd1.\
    groupby(col("item_name")).\
    agg(count(col("quantity")).alias("count_of_popularity"))

In [ ]:
pd2.show()

In [ ]:
pd2_inter=pd2.orderBy(col("count_of_popularity").desc()).limit(1)

In [ ]:
pd2_inter.select("item_name")

In [ ]:
data_list = pd2_inter.collect()

# Convert the list of Row objects to a list of dictionaries
data_dicts = [row.asDict() for row in data_list]

# Convert the list of dictionaries to a JSON string
json_data = json.dumps(data_dicts, indent=2)

# Print the JSON string
print(json_data)

In [ ]:
data_dicts[0]["item_name"]

In [ ]:
pd1.select(col("item_name"),col("choice_description")).filter(col("item_name")==data_dicts[0]["item_name"]).show(truncate=False)

In [ ]:
pd1.show()

In [ ]:
pd1.printSchema()

In [ ]:
default_value = 0.0
pd1_revised=pd1.withColumn("item_price_revised",when(cast(""FloatType()).isNull(), default_value)\
                              .otherwise(col("item_price").cast(FloatType()))).drop("item_price")

In [ ]:
pd1_revised.show()

In [ ]:
pd1_revised.printSchema()

Occupation code


In [ ]:
url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user"
file_path = "/tmp/users.tsv"
urllib.request.urlretrieve(url, file_path)

In [ ]:
# Read the CSV file with header and inferSchema options
path = '/user/sovik/practice/users.tsv'
users = spark.read.csv(path=path, header=True, inferSchema=True, sep='|')

# Add an index column
users_with_index = users.withColumn("index", monotonically_increasing_id())

In [ ]:
users_with_index.count()

In [ ]:
offset_val=users_with_index.count() -10

In [ ]:
offset_val

In [ ]:
users_with_index.offset(offset_val).show()

In [ ]:
len(users_with_index.columns)

In [ ]:
users_with_index.select(col("index")).show()

In [ ]:
users_with_index.printSchema()

In [ ]:
users_with_index.select(col("occupation")).distinct().count()

In [ ]:
users_with_index.groupBy(col("occupation")).\
    agg(count(col("occupation")).alias("count_of_occupation")).orderBy(col("count_of_occupation").desc()).limit(1).show()

In [ ]:
users_with_index.summary

In [ ]:
users_with_index.agg(round((sum(col("age"))/count("*")),2).alias("avg")).show()

In [ ]:
df=users_with_index.groupBy(col("occupation")).\
    agg(count(col("occupation")).alias("count_of_occupation")).orderBy(col("count_of_occupation").desc())



In [ ]:
df_min=df.select(min(col("count_of_occupation")).alias("min_val")).show()

In [ ]:
# Collect the result to get it as a list of Row objects
min_val_row = df_min.collect()

# Extract the minimum value from the Row object
min_val = min_val_row[0]['min_val']

# Print the minimum value
print("Minimum value of count_of_occupation:", min_val)

In [ ]:
data_list

In [ ]:
type(users_with_index.columns[3])

In [ ]:
# Sample list
d1 = [1, 2, 3, 4, 6, 4]

# Create a DataFrame from the list using lit function
df_explode = explode(d1)

# Explode the DataFrame


# Show the exploded DataFrame
df_explode.show()

filtering

In [4]:
chipotle=spark.read.\
    format("csv").\
        option("header","true").\
            option("inferschema","true").\
                option("sep","\t").\
                    load("/user/sovik/practice/chipotle.tsv")

In [5]:
chipotle.show()

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
|       5|       1| Chips and Guacamole|           

In [6]:
chipotle.filter(col("item_price") "").count()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1294107939.py, line 1)

In [7]:
default_value=0.0
chipotle_dedolarified=chipotle.withColumn("item_price_dedolarified",regexp_replace(col("item_price"),"^\$",""))
                              

In [8]:
chipotle_dedolarified.show()

+--------+--------+--------------------+--------------------+----------+-----------------------+
|order_id|quantity|           item_name|  choice_description|item_price|item_price_dedolarified|
+--------+--------+--------------------+--------------------+----------+-----------------------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |                  2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |                  3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |                  3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |                  2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |                 16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |                 10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |                  1.69 |
|       4|       1|       Stea

In [9]:
chipotle_updated=chipotle_dedolarified.withColumn("item_price_floatified",col("item_price_dedolarified").cast("float"))

In [10]:
dfv1=chipotle_updated.drop(col("item_price_dedolarified"))

In [11]:
df_v2=dfv1.drop(col("item_price"))

In [12]:
df_v2.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price_floatified: float (nullable = true)



Query 1

In [23]:
df_schema=df_v2.schema
field_tyles={}

if isinstance(df_schema,StructType):

    for field in df_schema:
        if field.dataType in field_tyles:
            field_tyles[field.dataType].append(field.name)
        else:
            field_tyles[field.dataType]=[field.name]
            
print(field_tyles)
count=0

for type,value in field_tyles.items():
    df= df_v2.select(*value)
    df.show()

{IntegerType(): ['order_id', 'quantity'], StringType(): ['item_name', 'choice_description'], FloatType(): ['item_price_floatified']}


Query2

+--------+--------+
|order_id|quantity|
+--------+--------+
|       1|       1|
|       1|       1|
|       1|       1|
|       1|       1|
|       2|       2|
|       3|       1|
|       3|       1|
|       4|       1|
|       4|       1|
|       5|       1|
|       5|       1|
|       6|       1|
|       6|       1|
|       7|       1|
|       7|       1|
|       8|       1|
|       8|       1|
|       9|       1|
|       9|       2|
|      10|       1|
+--------+--------+
only showing top 20 rows

+--------------------+--------------------+
|           item_name|  choice_description|
+--------------------+--------------------+
|Chips and Fresh T...|                NULL|
|                Izze|        [Clementine]|
|    Nantucket Nectar|             [Apple]|
|Chips and Tomatil...|                NULL|
|        Chicken Bowl|[Tomatillo-Red Ch...|
|        Chicken Bowl|[Fresh Tomato Sal...|
|       Side of Chips|                NULL|
|       Steak Burrito|[Tomatillo Red Ch...|
|    Steak

In [ ]:
df_v2.filter(col("item_price_floatified")>10).count()

In [ ]:
df_v2.filter(col("item_price_floatified")<10).count()

In [ ]:
df_v2.select(col("quantity")).orderBy(col("item_price_floatified").desc()).limit(1).show()

In [ ]:
euro=spark.read.format("csv").\
    option("header","true").\
        option("inferschema","true").\
            load("/user/sovik/practice/Euro_2012_stats_TEAM.csv")

In [ ]:
euro.show()

In [ ]:
len(euro.columns)

In [ ]:
disipline=euro.select('Team', 'Yellow Cards', 'Red Cards')

In [ ]:
disipline.show()

Select all columns except the last 3.

In [ ]:
df_euro=euro.columns

In [ ]:
column=df_euro[:3]

In [ ]:
column

In [ ]:
df=euro.select(*column)

In [ ]:


df.show()

In [ ]:
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'],
            'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'],
            'deaths': [523, 52, 25, 616, 43, 234, 523, 62, 62, 73, 37, 35],
            'battles': [5, 42, 2, 2, 4, 7, 8, 3, 4, 7, 8, 9],
            'size': [1045, 957, 1099, 1400, 1592, 1006, 987, 849, 973, 1005, 1099, 1523],
            'veterans': [1, 5, 62, 26, 73, 37, 949, 48, 48, 435, 63, 345],
            'readiness': [1, 2, 3, 3, 2, 1, 2, 3, 2, 1, 2, 3],
            'armored': [1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1],
            'deserters': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
            'origin': ['Arizona', 'California', 'Texas', 'Florida', 'Maine', 'Iowa', 'Alaska', 'Washington', 'Oregon', 'Wyoming', 'Louisana', 'Georgia']}

In [ ]:
columns=raw_data.keys()

In [ ]:
columns

In [ ]:
list_keys

In [ ]:
army=spark.read.json(spark.sparkContext.parallelize(raw_data))

In [ ]:
army.regiment.show()

In [ ]:
data_rows = [Row(**{key: value[i] for key, value in raw_data.items()}) for i in range(len(raw_data['regiment']))]

# Create DataFrame from list of Rows
df = spark.createDataFrame(data_rows)

# Show DataFrame
df.show()

In [ ]:
data=[('Alice','Badminton,Tennis'),\
    ('Bob','Cricket,Badminton,Tennis'),\
        ('Ralph','Cricket,Badminton,Tennis')]

columns=["Names","Hobbies"]

people=spark.createDataFrame(data,columns)

In [ ]:
people.show()

In [ ]:
people_v1=people.withColumn("Hobbies_array",split(col("Hobbies"),",")).drop("Hobbies")
#people_v2=people.select(col("Names"),explode(col("Hobbies")))

In [ ]:
people_v1.show()

In [ ]:
people_v2=people_v1.select(col("Names"),explode(col("Hobbies_array")))

In [ ]:
people_v2.show()

In [ ]:
data_city=[('Alice',"","AP"),\
    ('','AP',None),\
        (None,'','Bglr')]

column_city=["City1","City2","City3"]

In [ ]:
df_city=spark.createDataFrame(data_city,column_city)
df_city.show()

In [ ]:
df_city_v2=df_city.withColumn("result_string",concat_ws(",",col("City1"),col("City2"),col("City3")))

In [ ]:
df_city_v3=df_city_v2.withColumn("result_array",split(col("result_string"),",")).drop(col("result_string"))

In [ ]:
df_city_v3.show()

In [ ]:
df_city_vres=df_city_v3.select(explode(col("result_array")).alias("result"))

In [ ]:
df_city_vres.select(col("result")).filter(col("result")!="").distinct().show()

In [ ]:
data1=[(1,"Steve"),(2,"David"),(3,"John"),(4,"Shree"),(5,"Helen")]
data2=[(1,"SQL",90),(1,"PySpark",100),(2,"SQL",70),(2,"PySpark",60),(3,"SQL",30),(3,"PySpark",20),(4,"SQL",50),(4,"PySpark",50),(5,"SQL",45),(5,"PySpark",45)]

schema1=["Id","Name"]
schema2=["Id","Subject","Mark"]

df1=spark.createDataFrame(data1,schema1)
df2=spark.createDataFrame(data2,schema2)

In [ ]:
df1.show()

In [ ]:
df2.show()

In [ ]:
number_of_subjects=df2.select(col("Subject")).distinct().count()

In [ ]:
number_of_subjects

In [ ]:
df_joined=df1.join(df2,df1["Id"] == df2["Id"],how="left").select(df1["Id"], df1["Name"], df2["Subject"], df2["Mark"])
#df_result = df_joined.

In [ ]:
df_joined.show()

In [ ]:
df_joined_interim=df_joined.groupBy(col("Id"),col("Name")).agg((sum(col("Mark")/number_of_subjects)).alias("Percentage"))

In [ ]:
df_joined_interim.show()

In [ ]:
df_joined_interim.withColumn("Grade",when(col("Percentage")>90,"DIstinction").\
                                    when((col("Percentage")<=90) & (col("Percentage")>=50),"Average").\
                                            otherwise("Fail")).show()

In [ ]:
data1=[(1,"A",1000,"IT"),(2,"B",1500,"IT"),(3,"C",2500,"IT"),(4,"D",3000,"HR"),(5,"E",2000,"HR"),(6,"F",1000,"HR")
       ,(7,"G",4000,"Sales"),(8,"H",4000,"Sales"),(9,"I",1000,"Sales"),(10,"J",2000,"Sales")]
schema1=["EmpId","EmpName","Salary","DeptName"]
df=spark.createDataFrame(data1,schema1)
display(df)

In [ ]:
df.show()

In [ ]:
windowSpec=Window.partitionBy(col("DeptName")).orderBy(col("Salary").desc())

df.\
    select(col("EmpId"),col("EmpName"),col("Salary"),col("DeptName")).\
        withColumn("rank",rank().over(windowSpec)).\
            filter(col("rank")==1).\
                orderBy(col("Salary")).\
                    drop(col("rank")).show()

In [ ]:
data1=[(100,"Raj",None,1,"01-04-23",50000),
       (200,"Joanne",100,1,"01-04-23",4000),(200,"Joanne",100,1,"13-04-23",4500),(200,"Joanne",100,1,"14-04-23",4020)]
schema1=["EmpId","EmpName","Mgrid","deptid","salarydt","salary"]
df_salary=spark.createDataFrame(data1,schema1)
df_salary.show()
#department dataframe
data2=[(1,"IT"),
       (2,"HR")]
schema2=["deptid","deptname"]
df_dept=spark.createDataFrame(data2,schema2)
df_dept.show()


In [ ]:
df_salary_vmanager=df_salary

In [ ]:
df_joined=df_salary.\
    join(df_salary_vmanager,df_salary["EmpId"]==df_salary_vmanager["Mgrid"],how="left").\
        select(df_salary["EmpId"],df_salary["EmpName"],df_salary_vmanager["EmpName"].alias("Manager_name"),)

In [ ]:
df = spark.sql("SELECT current_date()-1 AS current_date")

In [ ]:
df.show()

In [ ]:
df1=df.collect()

In [ ]:
date_str=str(df1[0][0])

In [ ]:
new_df=spark.read.\
    format("csv").\
        option("header","true").\
            option("inferschema","true").\
                option("sep","\t").\
                    option("modifiedbefore",date_str).\
                    load("/user/sovik/practice/chipotle.tsv")

In [ ]:
new_df.show()

In [6]:
data=[(1,'2024-01-01',"I1",10,1000),(2,"2024-01-15","I2",20,2000),(3,"2024-02-01","I3",10,1500),(4,"2024-02-15","I4",20,2500),(5,"2024-03-01","I5",30,3000),(6,"2024-03-10","I6",40,3500),(7,"2024-03-20","I7",20,2500),(8,"2024-03-30","I8",10,1000)]
schema=["SOId","SODate","ItemId","ItemQty","ItemValue"]
df1=spark.createDataFrame(data,schema)
df1.show()

+----+----------+------+-------+---------+
|SOId|    SODate|ItemId|ItemQty|ItemValue|
+----+----------+------+-------+---------+
|   1|2024-01-01|    I1|     10|     1000|
|   2|2024-01-15|    I2|     20|     2000|
|   3|2024-02-01|    I3|     10|     1500|
|   4|2024-02-15|    I4|     20|     2500|
|   5|2024-03-01|    I5|     30|     3000|
|   6|2024-03-10|    I6|     40|     3500|
|   7|2024-03-20|    I7|     20|     2500|
|   8|2024-03-30|    I8|     10|     1000|
+----+----------+------+-------+---------+



In [7]:
df1_v1=df1.withColumn("Year_Month",date_format(col("SODate"),"MM-yyyy"))

In [8]:
df1_v2=df1_v1.drop(col("SODate"))
df1_v3=df1_v2.groupBy(col("Year_Month")).agg(sum(col("ItemValue")).alias("Tot_Sale"))

In [12]:
window_spec=Window.orderBy("Tot_Sale")

df1_v4=df1_v3.withColumn("Previous_month_tot_sal",lag(col("Tot_Sale"),1,0).over(window_spec))

In [13]:
df1_v5=df1_v4.withColumn("percrentage_diff",round(
        when(col("Previous_month_tot_sal") !=0, 
             ((col("Tot_Sale") - col("Previous_month_tot_sal")) * 100) / col("Previous_month_tot_sal"))
        .otherwise("NA"), 2
    )).drop(col("Previous_month_tot_sal"))

In [14]:
df1_v5.show()

24/06/05 17:18:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/05 17:18:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/05 17:18:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/05 17:18:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/05 17:18:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/05 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/06/05 1

+----------+--------+----------------+
|Year_Month|Tot_Sale|percrentage_diff|
+----------+--------+----------------+
|   01-2024|    3000|            NULL|
|   02-2024|    4000|           33.33|
|   03-2024|   10000|           150.0|
+----------+--------+----------------+



In [5]:
data=[(0,0,'start',0.712),(0,0,'end',1.520),(0,1,'start',3.140),(0,1,'end',4.120),
      (1,0,'start',0.550),(1,0,'end',1.550),(1,1,'start',0.430),(1,1,'end',1.420),
      (2,0,'start',4.100),(2,0,'end',4.512),(2,1,'start',2.500),(2,1,'end',5.000)]
schema=["Machine_id","processid","activityid","timestamp"]
df1=spark.createDataFrame(data,schema)
df1.show()

+----------+---------+----------+---------+
|Machine_id|processid|activityid|timestamp|
+----------+---------+----------+---------+
|         0|        0|     start|    0.712|
|         0|        0|       end|     1.52|
|         0|        1|     start|     3.14|
|         0|        1|       end|     4.12|
|         1|        0|     start|     0.55|
|         1|        0|       end|     1.55|
|         1|        1|     start|     0.43|
|         1|        1|       end|     1.42|
|         2|        0|     start|      4.1|
|         2|        0|       end|    4.512|
|         2|        1|     start|      2.5|
|         2|        1|       end|      5.0|
+----------+---------+----------+---------+



In [67]:
df1.printSchema()

root
 |-- Machine_id: long (nullable = true)
 |-- processid: long (nullable = true)
 |-- activityid: string (nullable = true)
 |-- timestamp: double (nullable = true)



In [65]:
df1_start=df1.filter(col("activityid")=='start')
df1_end=df1.filter(col("activityid")=='end')

df1_start.show()
df1_end.show()

+----------+---------+----------+---------+
|Machine_id|processid|activityid|timestamp|
+----------+---------+----------+---------+
|         0|        0|     start|    0.712|
|         0|        1|     start|     3.14|
|         1|        0|     start|     0.55|
|         1|        1|     start|     0.43|
|         2|        0|     start|      4.1|
|         2|        1|     start|      2.5|
+----------+---------+----------+---------+

+----------+---------+----------+---------+
|Machine_id|processid|activityid|timestamp|
+----------+---------+----------+---------+
|         0|        0|       end|     1.52|
|         0|        1|       end|     4.12|
|         1|        0|       end|     1.55|
|         1|        1|       end|     1.42|
|         2|        0|       end|    4.512|
|         2|        1|       end|      5.0|
+----------+---------+----------+---------+



In [72]:
df1_start=df1.filter(col("activityid")=='start')
df1_end=df1.filter(col("activityid")=='end')


# Join the two DataFrames on processid
df_joined = df1_end.join(df1_start, (df1_end["processid"] == df1_start["processid"]) &(df1_end["Machine_id"] == df1_start["Machine_id"]), how="inner") \
    .select(df1_end["Machine_id"], df1_end["processid"], df1_end["timestamp"].alias("end_timestamp"), df1_start["timestamp"].alias("start_timestamp"))

# Calculate the activity run time
df_result = df_joined.withColumn("activity_run", col("end_timestamp") - col("start_timestamp"))

# Show the result
df_result.show()

AnalysisException: Column timestamp#591 are ambiguous. It's probably because you joined several Datasets together, and some of these Datasets are the same. This column points to one of the Datasets but Spark is unable to figure out which one. Please alias the Datasets with different names via `Dataset.as` before joining them, and specify the column using qualified name, e.g. `df.as("a").join(df.as("b"), $"a.id" > $"b.id")`. You can also set spark.sql.analyzer.failAmbiguousSelfJoin to false to disable this check.

In [75]:
df1_start = df1.filter(col("activityid") == 'start').alias("start")
df1_end = df1.filter(col("activityid") == 'end').alias("end")

# Join the two DataFrames on processid
df_joined = df1_end.join(df1_start, df1_end["processid"] == df1_start["processid"], how="inner") \
    .select(df1_end["Machine_id"], df1_end["processid"], df1_end["end"], df1_start["start"])

# Calculate the activity run time
df_result = df_joined.withColumn("activity_run", col("end") - col("start"))

# Show the result
df_result.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `end` cannot be resolved. Did you mean one of the following? [`Machine_id`, `processid`, `activityid`, `timestamp`].

In [81]:
df1_start = df1.filter(col("activityid") == 'start').select("*",col("timestamp").alias("start")).drop(col("timestamp"))
df1_end = df1.filter(col("activityid") == 'end').select("*",col("timestamp").alias("end")).drop(col("timestamp"))

df1_start.show()
df1_end.show()

+----------+---------+----------+-----+
|Machine_id|processid|activityid|start|
+----------+---------+----------+-----+
|         0|        0|     start|0.712|
|         0|        1|     start| 3.14|
|         1|        0|     start| 0.55|
|         1|        1|     start| 0.43|
|         2|        0|     start|  4.1|
|         2|        1|     start|  2.5|
+----------+---------+----------+-----+

+----------+---------+----------+-----+
|Machine_id|processid|activityid|  end|
+----------+---------+----------+-----+
|         0|        0|       end| 1.52|
|         0|        1|       end| 4.12|
|         1|        0|       end| 1.55|
|         1|        1|       end| 1.42|
|         2|        0|       end|4.512|
|         2|        1|       end|  5.0|
+----------+---------+----------+-----+



In [86]:
# Join the two DataFrames on processid
df_joined = df1_end.join(df1_start, (df1_end["processid"] == df1_start["processid"])&(df1_end["Machine_id"] == df1_start["Machine_id"]), how="inner") \
    .select(df1_end["Machine_id"], df1_end["processid"], df1_end["end"], df1_start["start"])

# Calculate the activity run time
df_result = df_joined.withColumn("activity_run", col("end") - col("start"))

# Show the result
df_result.groupBy(col("Machine_id")).agg(avg(col("activity_run").alias("avg_activity_run"))).orderBy(col("Machine_id")).show()

+----------+-------------------------------------+
|Machine_id|avg(activity_run AS avg_activity_run)|
+----------+-------------------------------------+
|         0|                                0.894|
|         1|                                0.995|
|         2|                                1.456|
+----------+-------------------------------------+



In [87]:
data=[(1,'John','ADF'),(1,'John','ADB'),(1,'John','PowerBI'),(2,'Joanne','ADF'),(2,'Joanne','SQL'),(2,'Joanne','Crystal Report'),(3,'Vikas','ADF'),(3,'Vikas','SQL'),(3,'Vikas','SSIS'),(4,'Monu','SQL'),(4,'Monu','SSIS'),(4,'Monu','SSAS'),(4,'Monu','ADF')]
schema=["EmpId","EmpName","Skill"]
df1=spark.createDataFrame(data,schema)
display(df1)

DataFrame[EmpId: bigint, EmpName: string, Skill: string]

In [88]:
df1.show()

+-----+-------+--------------+
|EmpId|EmpName|         Skill|
+-----+-------+--------------+
|    1|   John|           ADF|
|    1|   John|           ADB|
|    1|   John|       PowerBI|
|    2| Joanne|           ADF|
|    2| Joanne|           SQL|
|    2| Joanne|Crystal Report|
|    3|  Vikas|           ADF|
|    3|  Vikas|           SQL|
|    3|  Vikas|          SSIS|
|    4|   Monu|           SQL|
|    4|   Monu|          SSIS|
|    4|   Monu|          SSAS|
|    4|   Monu|           ADF|
+-----+-------+--------------+



In [94]:
df1_v1=df1.groupBy(col("EmpName")).agg(collect_list(col("Skill")).alias("Skill_list"))
df1_v1.withColumn("Skill_list_updt",concat_ws(',',col("Skill_list"))).drop(col("Skill_list")).show(truncate=False)

+-------+----------------------+
|EmpName|Skill_list_updt       |
+-------+----------------------+
|John   |ADF,ADB,PowerBI       |
|Joanne |ADF,SQL,Crystal Report|
|Monu   |SQL,SSIS,SSAS,ADF     |
|Vikas  |ADF,SQL,SSIS          |
+-------+----------------------+



In [8]:
data={
  "name": "John Doe",
  "age": 30,
  "address": {
    "street": "123 Main St",
    "city": "Anytown",
    "state": "CA",
    "postalCode": "12345"
  },
  "contactDetails": [
    {
      "type": "home",
      "number": "555-555-5555"
    },
    {
      "type": "work",
      "number": "555-555-5556"
    },
    {
      "email": "johndoe@example.com"
    }
  ],
  "projects": [
    {
      "name": "Project A",
      "deadline": "2024-12-31"
    },
    {
      "name": "Project B",
      "deadline": "2025-06-30"
    }
  ]
}

json_data=json.dumps(data)

In [10]:
df = spark.read.json(spark.sparkContext.parallelize([json_data]))

In [12]:
df.show(truncate=False)

+---------------------------------+---+-------------------------------------------------------------------------------------------+--------+--------------------------------------------------+
|address                          |age|contactDetails                                                                             |name    |projects                                          |
+---------------------------------+---+-------------------------------------------------------------------------------------------+--------+--------------------------------------------------+
|{Anytown, 12345, CA, 123 Main St}|30 |[{NULL, 555-555-5555, home}, {NULL, 555-555-5556, work}, {johndoe@example.com, NULL, NULL}]|John Doe|[{2024-12-31, Project A}, {2025-06-30, Project B}]|
+---------------------------------+---+-------------------------------------------------------------------------------------------+--------+--------------------------------------------------+



In [13]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- postalCode: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- street: string (nullable = true)
 |-- age: long (nullable = true)
 |-- contactDetails: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- number: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- projects: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- deadline: string (nullable = true)
 |    |    |-- name: string (nullable = true)



Name,address,projects

In [15]:
df_v1=df.select(col("name"),col("address"),col("projects"))

In [16]:
df_v1.show(truncate=False)

+--------+---------------------------------+--------------------------------------------------+
|name    |address                          |projects                                          |
+--------+---------------------------------+--------------------------------------------------+
|John Doe|{Anytown, 12345, CA, 123 Main St}|[{2024-12-31, Project A}, {2025-06-30, Project B}]|
+--------+---------------------------------+--------------------------------------------------+



In [19]:
df_v2=df_v1.select("*").withColumn("projects_exploded",explode(col("projects"))).drop(col("projects"))

In [21]:
df_v2.printSchema()

root
 |-- name: string (nullable = true)
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- postalCode: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- street: string (nullable = true)
 |-- projects_exploded: struct (nullable = true)
 |    |-- deadline: string (nullable = true)
 |    |-- name: string (nullable = true)



In [26]:
df_v3=df_v2.select("*").withColumn("deadline",df_v2['projects_exploded']['deadline']).drop(col("projects_exploded"))

In [28]:
df_v3.printSchema()

root
 |-- name: string (nullable = true)
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- postalCode: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- street: string (nullable = true)
 |-- deadline: string (nullable = true)



In [32]:
data2=[
  {
    "name": "John Doe",
    "age": 30,
    "address": {
      "street": "123 Main St",
      "city": "Anytown",
      "state": "CA",
      "postalCode": "12345"
    },
    "contactDetails": [
      {
        "type": "home",
        "number": "555-555-5555"
      },
      {
        "type": "work",
        "number": "555-555-5556"
      },
      {
        "email": "johndoe@example.com"
      }
    ],
    "projects": [
      {
        "name": "Project A",
        "deadline": "2024-12-31"
      },
      {
        "name": "Project B",
        "deadline": "2025-06-30"
      }
    ]
  },
  {
    "name": "Jane Smith",
    "age": 28,
    "address": {
      "street": "456 Elm St",
      "city": "Othertown",
      "state": "NY",
      "postalCode": "67890"
    },
    "contactDetails": [
      {
        "type": "home",
        "number": "555-555-1234"
      },
      {
        "type": "work",
        "number": "555-555-5678"
      },
      {
        "email": "janesmith@example.com"
      }
    ],
    "projects": [
      {
        "name": "Project C",
        "deadline": "2025-03-15"
      },
      {
        "name": "Project D",
        "deadline": "2025-08-22"
      }
    ]
  }
]
json_data2 = json.dumps(data2)

# Create a DataFrame from the JSON string
df2 = spark.read.json(spark.sparkContext.parallelize([json_data2]))


In [33]:
df2.show()

+--------------------+---+--------------------+----------+--------------------+
|             address|age|      contactDetails|      name|            projects|
+--------------------+---+--------------------+----------+--------------------+
|{Anytown, 12345, ...| 30|[{NULL, 555-555-5...|  John Doe|[{2024-12-31, Pro...|
|{Othertown, 67890...| 28|[{NULL, 555-555-1...|Jane Smith|[{2025-03-15, Pro...|
+--------------------+---+--------------------+----------+--------------------+



In [34]:
df_c2r=spark.read.csv(path='/user/sovik/practice/col_2_row.csv',header=True,inferSchema=True)

In [35]:
df_c2r.show(truncate=True)

+--------------------+--------------------+-----------+---+--------------+------------------------+------------------------+-------------------------+-------------------------+
|             svoc_id|        token_number|dealer_code|lob|transaction_no|transaction_type_booking|transaction_type_invoice|transaction_type_delivery|transaction_type_job_card|
+--------------------+--------------------+-----------+---+--------------+------------------------+------------------------+-------------------------+-------------------------+
|2072e38e-192f-49a...|23d89455-30eb-4d3...| LQSDEALER1|  1|   OB202300005|                 booking|                 invoice|             deliverynote|                 job_card|
|61fd04b4-f6bf-474...|d0e52e2d-867b-43b...| LQSDEALER1|  1|   OB202300005|                 booking|                 invoice|             deliverynote|                 job_card|
|933d804a-b966-427...|d0e52e2d-867b-43b...| LQSDEALER1|  1|   OB202300005|                 booking|                

SELECT svoc_id,token_number,dealer_code,lob,transaction_no,transaction_type FROM (SELECT svoc_id,token_number,dealer_code,lob,transaction_no,stack(4,'transaction_type_booking', transaction_type_booking,'transaction_type_invoice', transaction_type_invoice,'transaction_type_delivery', transaction_type_delivery,'transaction_type_jobcart', transaction_type_jobcart) AS (type_category, transaction_type) FROM pivot_transaction_type) AS unpiv

In [36]:
df_c2r.createOrReplaceTempView("transition_type")

In [41]:
current_database

<function pyspark.sql.functions.current_database() -> pyspark.sql.column.Column>

In [43]:
databases = spark.catalog.listDatabases()
print("Databases:")
for db in databases:
    print(db.name)

Databases:
default
nyse_db
retail_db


In [44]:
spark.sql("use retail_db")

DataFrame[]

In [42]:
df_c2r.

DataFrame[current_database(): string]

In [45]:
current_database = spark.catalog.currentDatabase()
print(f"Current Database after change: {current_database}")

Current Database after change: retail_db


In [46]:
df_c2r.createOrReplaceTempView("transition_type_1")

In [48]:
spark.sql("select * from transition_type_1").show(truncate=False)

+------------------------------------+------------------------------------+-----------+---+--------------+------------------------+------------------------+-------------------------+-------------------------+
|svoc_id                             |token_number                        |dealer_code|lob|transaction_no|transaction_type_booking|transaction_type_invoice|transaction_type_delivery|transaction_type_job_card|
+------------------------------------+------------------------------------+-----------+---+--------------+------------------------+------------------------+-------------------------+-------------------------+
|2072e38e-192f-49a6-ac7b-64fec86b5ac3|23d89455-30eb-4d3c-b492-5f1545679d02|LQSDEALER1 |1  |OB202300005   |booking                 |invoice                 |deliverynote             |job_card                 |
|61fd04b4-f6bf-474a-8ddc-620fd73f2577|d0e52e2d-867b-43bc-ae59-5dc370144ed7|LQSDEALER1 |1  |OB202300005   |booking                 |invoice                 |delivery

In [50]:
spark.sql("""
          SELECT svoc_id,token_number,dealer_code,lob,transaction_no,transaction_type FROM (SELECT svoc_id,token_number,dealer_code,lob,transaction_no,stack(4,'transaction_type_booking', transaction_type_booking,'transaction_type_invoice', transaction_type_invoice,'transaction_type_delivery', transaction_type_delivery,'transaction_type_jobcart', transaction_type_jobcart) AS (type_category, transaction_type) FROM transition_type_1) AS unpiv         
          """).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `transaction_type_jobcart` cannot be resolved. Did you mean one of the following? [`transaction_type_job_card`, `transaction_type_booking`, `transaction_type_invoice`, `transaction_type_delivery`, `transaction_no`].; line 2 pos 348;
'Project ['svoc_id, 'token_number, 'dealer_code, 'lob, 'transaction_no, 'transaction_type]
+- 'SubqueryAlias unpiv
   +- 'Project [svoc_id#255, token_number#256, dealer_code#257, lob#258, transaction_no#259, 'stack(4, transaction_type_booking, transaction_type_booking#260, transaction_type_invoice, transaction_type_invoice#261, transaction_type_delivery, transaction_type_delivery#262, transaction_type_jobcart, 'transaction_type_jobcart) AS (type_category, transaction_type)]
      +- SubqueryAlias transition_type_1
         +- View (`transition_type_1`, [svoc_id#255,token_number#256,dealer_code#257,lob#258,transaction_no#259,transaction_type_booking#260,transactio

In [4]:
pd1=spark.read.\
    parquet("/user/sovik/nag_issues/20240608-114546125.parquet")

In [5]:
pd1.show()

+------------------+-------------------+--------------------+-----------------------+-----------------+----------------------+-------------------+----------------------+--------------------+--------------------+
|header__change_seq|header__change_oper| header__change_mask|header__stream_position|header__operation|header__transaction_id|  header__timestamp|header__partition_name|                 _id|                _doc|
+------------------+-------------------+--------------------+-----------------------+-----------------+----------------------+-------------------+----------------------+--------------------+--------------------+
|2.0240607114526E34|                  I|\x03\x00\x00\x00\...|   245;6385335753204...|           INSERT|  4155544F434F4D4D4...|2024-06-07 11:45:26|  20240607T093000_2...|6662f2d6ed500c821...|{"_id": {"$oid": ...|
+------------------+-------------------+--------------------+-----------------------+-----------------+----------------------+-------------------+------

In [7]:
pd1_schema=pd1.schema

In [9]:
pd1_schema

StructType([StructField('header__change_seq', DoubleType(), True), StructField('header__change_oper', StringType(), True), StructField('header__change_mask', StringType(), True), StructField('header__stream_position', StringType(), True), StructField('header__operation', StringType(), True), StructField('header__transaction_id', StringType(), True), StructField('header__timestamp', TimestampNTZType(), True), StructField('header__partition_name', StringType(), True), StructField('_id', StringType(), True), StructField('_doc', StringType(), True)])

In [12]:
schema_list=list(pd1_schema)

In [17]:
last_col=schema_list[-1].name

In [18]:
last_col

'_doc'

In [22]:
df_json=pd1.select(col(last_col))

In [23]:
df_json.show()

+--------------------+
|                _doc|
+--------------------+
|{"_id": {"$oid": ...|
+--------------------+



In [25]:
json_value=df_json.toJSON()

In [29]:
json_value=json_value.collect()

In [30]:
json_value

['{"_doc":"{\\"_id\\": {\\"$oid\\": \\"6662f2d6ed500c821fd0f877\\"}, \\"outlet_id\\": \\"002\\", \\"Email\\": \\"sagar.daware@suzuki-digital.com\\", \\"first_name\\": \\"Sagar \\", \\"last_name\\": \\"Daware\\", \\"communication_medium_id\\": \\"1663751859054\\", \\"Phone\\": \\"8888657594\\", \\"lead_source_id\\": \\"1663926070773\\", \\"lead_source\\": \\"Website\\", \\"lead_mode_id\\": \\"1710746578356\\", \\"lead_mode\\": \\"Test Drive Form\\", \\"outlet_name\\": \\"RANA SUZUKI\\", \\"client_id\\": 450, \\"parent_dealer_id\\": \\"LQS-ADMIN\\", \\"parent_dealer_name\\": \\"LQS-ADMIN\\", \\"distributor_code\\": \\"DIS001\\", \\"outlet_code\\": \\"002\\", \\"is_deleted\\": 0, \\"qualified\\": 1, \\"auto_qualified\\": 1, \\"duplicate\\": 1, \\"different_channel\\": 0, \\"lead_type\\": \\"Web\\", \\"lead_type_id\\": \\"1663926070773\\", \\"utm_source\\": \\"direct\\", \\"utm_source_id\\": \\"1717588141715\\", \\"utm_medium\\": \\"\\", \\"lead_status\\": \\"Cold\\", \\"status\\": \\"1663

In [33]:
cleaned_data = [json.loads(json.loads(record)[last_col].replace('\\"', '"')) for record in json_value]

In [36]:
cleansed_last_col=spark.read.json(spark.sparkContext.parallelize([cleaned_data]))

In [37]:
cleansed_last_col.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- auto_qualified: long (nullable = true)
 |-- business_id: string (nullable = true)
 |-- business_type: string (nullable = true)
 |-- client_id: long (nullable = true)
 |-- communication_medium_id: string (nullable = true)
 |-- consent_created_using: string (nullable = true)
 |-- different_channel: long (nullable = true)
 |-- distributor_code: string (nullable = true)
 |-- duplicate: long (nullable = true)
 |-- email_reminder_date: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- enquiry_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- form_name: string (nullable = true)
 |-- is_deleted: long (nullable = true)
 |-- is_home_visit_done: long (nullable = true)
 |-- is_prebooking_done: long (nullable = true)
 |-- is_quotation_done: long (nullable = true)
 |-- is_show_room_visit_do

In [6]:
spark.stop()